In [10]:
import numpy as np
import pandas as pd

# Format a set of arrivals for testing

In [32]:
n_events = 5
lat_min, lat_max = 35, 36
lon_min, lon_max = -118, -117
depth_min, depth_max = 0, 30

with pd.HDFStore("/home/malcolmw/google_drive/malcolm.white@usc.edu/data/events/scedc/h5/scedc_2018.h5") as store:
    events = store["events"]
    arrivals = store["arrivals"]
    
events = events[
     (events["lat"] > lat_min)
    &(events["lat"] < lat_max)
    &(events["lon"] > lon_min)
    &(events["lon"] < lon_max)
    &(events["depth"] > depth_min)
    &(events["depth"] < depth_max)
]

event_ids = np.random.choice(events["event_id"], n_events)
arrivals = arrivals.set_index("event_id").loc[event_ids]
split = arrivals["station_id"].str.split(".", expand=True)
arrivals["network"], arrivals["station"] = split[0], split[1]
arrivals = arrivals.drop(columns=["station_id"])
arrivals = arrivals.reset_index(drop=True)

with pd.HDFStore("data/small_test/detections.h5") as store:
    store["detections"] = arrivals

# Format network geometry for testing

In [52]:
with pd.HDFStore("/home/malcolmw/google_drive/malcolm.white@usc.edu/data/networks/scsn/stations_simple.h5") as store:
    stations = store["stations"]
    
stations = stations.rename(columns={"net": "network", "sta": "station", "lat": "latitude", "lon": "longitude", "elev": "elevation"})
stations = stations.set_index(["network", "station"])
stations = stations.loc[stations.index.intersection(arrivals.set_index(["network", "station"]).index)]
stations = stations.reset_index()

with pd.HDFStore("data/small_test/stations.h5") as store:
    store["stations"] = stations